In [2]:
from openai import OpenAI
import pprint
import time
client = OpenAI()
start_reading = time.time()
audio_file = open("memento-sample-audio.mp3", "rb")
end_reading = time.time()
print(f"Reading audio file took {end_reading - start_reading} seconds")

start_transcription = time.time()
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["segment"]
)
end_transcription = time.time()
print(f"Transcription took {end_transcription - start_transcription} seconds")

print(transcript)
pprint.pp(str(transcript))

Reading audio file took 0.01252603530883789 seconds
Transcription took 6.579843521118164 seconds
Transcription(text="Hey man, how's it going? Good, good. How about you? What's your favorite animal, man? Pretty good. My favorite animal is dinosaur. But let me start, like, what's your name? I'm Arush. What's yours? I'm Vig, by the way. Nice to meet you. So how do you like the conference? It's pretty cool. It's really big. I'm meeting a lot of really cool people. My favorite animal is the capybara, so I've been finding a lot of capybara lovers around here. It's been nice. Tell me more about capybara. They're just really goofy. Interesting. Is that why you like them? Yeah, yeah. It's... That's very interesting, man. Like, I've never seen someone who's a fan of capybara. It's the first time. Yeah, they're like huge rats, basically. Oh, okay. Interesting. They're really cute, and there's been a trend going on, going around on, like, Instagram and stuff of reels of capybaras doing random stuf